# welcome 


### Data description 
### Main dataset: 
### This dataset informs the medical situation of individuals segregation the observations by municipality, serializing them and providing income data of each one of them. Also, it provides an specific score about overall health & the case of an specific condition.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

url3 = r"Main_Dataset.csv"
md = pd.read_csv(url3)
url2 = r"Sociodemographics Data.csv"
sd = pd.read_csv(url2)
url1 = r"Municipality Pollution Data.csv"
mu = pd.read_csv(url1)


In [ ]:
# in case u want to always get the same sample use after the (n) after a coma random_state=: 
# This ensures that every time you run the code, you will get the same 500 rows sampled from your dataset (given the same input data).

md_AND_mu_merged = pd.merge(md, mu, on='Municipality')
md_r_sample = md_AND_mu_merged.sample(n=100,random_state=1)


fig = px.scatter_3d(md_r_sample, 
                     z='Overall_Health_Score', 
                     y='Traffic_Pollution', 
                     x='Air_Quality_Index',
                     color='Overall_Health_Score', 
                     opacity=0.65, 
                     size_max=20,
                     #hover_data=['Alcohol_Consumer','Smoker',"Physical_Activity",'Diet_Quality'],
                     #  color_discrete_sequence=px.colors.qualitative.Dark24
)

fig.update_layout(
    title="3D Scatter Plot of Health Score in relation with Traffic Pollution and AirQuality Index (Sampled Data)",
    scene=dict(
        yaxis_title="Traffic Pollution",
        zaxis_title="Health Score",
        xaxis_title=" AirQuality Index",
        bgcolor="black"  
    ),
    paper_bgcolor="black",  
    font=dict(color="white"),  
    scene_camera=dict(eye=dict(x=1.8, y=1.8, z=0.8))
)
fig.show()



### 1. **IMC > 30, Dieta poco saludable, No realiza actividad física**  
- Podría revelar una conexión clara con **diabetes tipo 2**, **síndrome metabólico** y **enfermedades articulares**.  
- **Idea:** Analizar si el sedentarismo y la dieta son los principales factores desencadenantes o si el IMC alto es el factor dominante.  



In [ ]:

md_r_sample = md.sample(n=8000,random_state=1).dropna(subset=["Income","Physical_Activity","BMI","Diet_Quality","Overall_Health_Score"])
fig = px.scatter(
    md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor')& Medical_Condition != 'Healthy'"), 
    y="BMI", 
    x="Overall_Health_Score", 
    color="Medical_Condition",
    hover_name="Medical_Condition",  
    color_discrete_sequence=px.colors.diverging.Spectral
)

fig.show()


In [ ]:



random_states = [10, 20, 30, 40, 50 ]  
fig = make_subplots(rows=1, cols=5, subplot_titles=[f"random sample = {rs}" for rs in random_states])

for i, rs in enumerate(random_states):
    md_r_sample = md.sample(n=1000, random_state=rs).dropna(subset=["Income", "Physical_Activity", "BMI", "Diet_Quality", "Overall_Health_Score"])
    filtered_data = md_r_sample.query("(Physical_Activity == 'Low' | BMI > 30 | Diet_Quality == 'Poor')& Medical_Condition != 'Healthy'")


    scatter = px.scatter(
    filtered_data, y="BMI", x="Overall_Health_Score",
    color="Medical_Condition",
    hover_name="Medical_Condition",
    size_max=10,  
    color_discrete_sequence=px.colors.diverging.Spectral
    )

    for trace in scatter.data:
        fig.add_trace(trace, row=1, col=i + 1)
fig.update_layout(title="Scatter Plots with Different Random States", 
    showlegend=False)


fig.show()



### 2. **Consumo excesivo de alcohol, Fumador, IMC > 25**  
- La combinación de tabaquismo y alcohol podría aumentar el riesgo de **enfermedades hepáticas** y **cáncer**.  
- **Idea:** Examinar si el consumo excesivo de alcohol o el tabaquismo tiene un impacto mayor en el desarrollo de cáncer.  

In [ ]:
Alcohol_Smoker_BMI_25 = md[(md["BMI"] > 25) & (md["Alcohol_Consumer"] == 1) & (md["Smoker"] == 1)]
Cancer = md[md["Medical_Condition"] == "Cancer"]

#### medias a traves de diferentes municipalidades 

In [28]:

fig = px.box(mu, x="Municipality", y="Traffic_Pollution",  color="Municipality")
fig.show()

In [29]:

mu_sorted = mu.groupby("Municipality")["Traffic_Pollution"].median().sort_values().index


fig = px.box(mu, x="Municipality", y="Traffic_Pollution", color="Municipality")

fig.update_layout(
    title="Traffic Pollution Levels by Municipality",
    xaxis_title="Municipality",
    yaxis_title="Traffic Pollution Index",
    width=1200, height=600,  
    xaxis_tickangle=-75,  
    font=dict(size=14), 
    xaxis={'categoryorder':'array', 'categoryarray':mu_sorted}  )

fig.update_traces(marker=dict(opacity=0.8)) 
fig.show()


In [33]:
pollution_medians = mu.groupby("Municipality")["Traffic_Pollution"].median()

top_10_max = pollution_medians.nlargest(10).index
top_10_min = pollution_medians.nsmallest(10).index

mu_top_max = mu[mu["Municipality"].isin(top_10_max)]
mu_top_min = mu[mu["Municipality"].isin(top_10_min)]


fig_max = px.box(mu_top_max, x="Municipality", y="Traffic_Pollution",
                #color="Municipality",
                title="Top 10 Municipalities with Highest Traffic Pollution over the Years",
                #color_discrete_sequence=px.colors.qualitative.Set1
                )

fig_min = px.box(mu_top_min, x="Municipality", y="Traffic_Pollution",
                #color="Municipality",
                title="Top 10 Municipalities with Lowest Traffic Pollution over the Years",
                #color_discrete_sequence=px.colors.qualitative.Set2
                )

for fig in [fig_max, fig_min]:
    fig.update_layout(
        xaxis_title="Municipality",
        yaxis_title="Traffic Pollution Index",
        width=600, height=400,  
        xaxis_tickangle=-45, 
        font=dict(size=12),  
        showlegend=False  
    )
    fig.update_traces(marker=dict(opacity=0.8))  

fig_max.show()
fig_min.show()


In [ ]:

pollution_medians = mu.groupby("Municipality")["Traffic_Pollution"].median()

top_10_max = pollution_medians.nlargest(10)
top_10_min = pollution_medians.nsmallest(10)

mu_top_max = mu[mu["Municipality"].isin(top_10_max.index)]
mu_top_min = mu[mu["Municipality"].isin(top_10_min.index)]

viridis_colors = px.colors.sequential.Viridis

max_colors = {mun: viridis_colors[i] for i, mun in enumerate(top_10_max.index)}
min_colors = {mun: viridis_colors[i] for i, mun in enumerate(top_10_min.index)}


fig_max = px.box(mu_top_max, x="Municipality", y="Traffic_Pollution", color="Municipality",
                 title="Top 10 Municipalities with Highest Traffic Pollution",
                 category_orders={"Municipality": top_10_max.index})

fig_min = px.box(mu_top_min, x="Municipality", y="Traffic_Pollution", color="Municipality",
                 title="Top 10 Municipalities with Lowest Traffic Pollution",
                 category_orders={"Municipality": top_10_min.index})

# Apply manually assigned Viridis colors
fig_max.for_each_trace(lambda t: t.update(marker=dict(color=max_colors[t.name])))
fig_min.for_each_trace(lambda t: t.update(marker=dict(color=min_colors[t.name])))

for fig in [fig_max, fig_min]:
    fig.update_layout(
        xaxis_title="Municipality",
        yaxis_title="Traffic Pollution Index",
        width=600, height=400,
        xaxis_tickangle=-45,
        font=dict(size=12),
        showlegend=False  
    )
    fig.update_traces(marker=dict(opacity=0.8))  

fig_max.show()
fig_min.show()
